<a href="https://colab.research.google.com/github/pacozaa/LLM-Paper-To-Code/blob/main/Self_Consistency_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are utilizing free tier of GitHub Models.


Check out the billing condition here but you should have a free tier if you have GitHub account:

https://docs.github.com/en/github-models/use-github-models/prototyping-with-ai-models#rate-limits

In [86]:
#Install Dependency
!pip install azure-ai-inference datasets

In [87]:
import os
import time
import re
import json
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential
from collections import Counter
from google.colab import userdata
from datasets import load_dataset

We are using one of the puzzle from **ZebraLogic Bench** for this experiment

- Dataset:
https://huggingface.co/datasets/allenai/ZebraLogicBench-private/viewer/grid_mode/test?row=0&views%5B%5D=grid_mode
- Dataset Article:
https://huggingface.co/blog/yuchenlin/zebra-logic

- Find the example row by this sql in huggingface dataset studio
```sql
SELECT *
FROM grid_mode
WHERE id = 'lgp-test-3x3-24'
LIMIT 10;
```
- or use this link: https://huggingface.co/datasets/allenai/ZebraLogicBench-private/viewer/grid_mode/test?f%5Bsize%5D%5Bvalue%5D=%273*3%27&row=41&views%5B%5D=grid_mode&sql=SELECT+*+%0A++FROM+grid_mode+%0A++WHERE+id+%3D+%27lgp-test-3x3-24%27%0A++LIMIT+10%3B


# Start Coding

# 1. Set Up Parameters

In [88]:


#GitHub Token

# 2. Prepare Prompt

In [89]:
# Prompt
# CoT Prompt
system_prompt="""
 Each example/input/question is a Logic Grid Puzzle, also known as a Zebra Puzzle. In each puzzle, we are given N houses (numbered 1 to N from left to right) and M features for each house. There are N distinct values for each feature, and each house must have a unique value for each feature. Given a list of clues, one should be able to deduce a unique correct assignment of values. The logic grid puzzle is a typical Constraint Satisfaction Problem (CSP) and is often used to test humans' logical reasoning abilities in exams such as the Law School Admission Test (LSAT).

You always solve problem step by step
start answering with `## Reasoning steps:` and end with

## Final answer:
```json

````

NB. Always use double quote inside json block
"""
# Example Input Prompt
user_prompt="""
There are 2 houses, numbered 1 to 2 from left to right.
Each house is occupied by a different person.
Each house has a unique attribute for each of the following characteristics:
- Each person has a unique name: **Arnold, Eric**
- People own unique car models: **ford f150, tesla model 3**
- The people keep unique animals: **cat, horse**

**Clues**:
1. Eric is directly left of the person who owns a Tesla Model 3.
2. The person who keeps horses is in the first house.
"""
# Example Output Prompt
assistant_message="""
## Reasoning steps:

From Clue 1, we know that Eric is to the left of someone, so he must be the owner of House 1 because House 2 is the rightmost house.
Additionally, we know that the person in House 2 must be Arnold, and he owns a Tesla Model 3. Thus, Eric owns a Ford F150.
From Clue 2, we know that Eric keeps horses in House 1, which means the other house must keep cats. Finally, we arrive at the unique solution to this puzzle.
The solution is presented in table format:

## Final answer:

```json
{
   "header":[
      "Houses",
      "Name",
      "CarModel",
      "Animal"
   ],
   "rows":[
      [
         "1",
         "Eric",
         "ford f150",
         "horse"
      ],
      [
         "2",
         "Arnold",
         "tesla model 3",
         "cat"
      ]
   ]
}
```
"""
# lgp-test-2x2-33
# lgp-test-6x5-2
# lgp-test-2x4-33
# lgp-test-6x6-5
# lgp-test-3x3-24
question="""
Question:
There are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
- Each person has a unique name: `Peter`, `Eric`, `Arnold`
- Each person has a favorite color: `red`, `white`, `yellow`
- Each mother is accompanied by their child: `Fred`, `Meredith`, `Bella`

## Clues:
1. Arnold is the person whose favorite color is red.
2. The person's child is named Fred is somewhere to the left of Eric.
3. The person whose favorite color is red is in the second house.
4. The person's child is named Bella is in the first house.
5. The person who loves white is the person's child is named Meredith.

## Headers
"header": [
"House",
"Name",
"Color",
"Children"
]
"""

# 3. Construct Messages with Few Shot Prompts

# 4. Extract Functions

In [90]:

def extract_final_answer_regex(answer):
    match = re.search(r'## Final answer:\s*(.*)', answer, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

def extract_json_from_string(input_string):
    """
    Extract JSON from a string that contains ```json``` code blocks.

    Args:
        input_string (str): Input string that may contain JSON code blocks

    Returns:
        list: List of parsed JSON objects/dictionaries, or empty list if none found
    """
    # Regular expression pattern to match JSON code blocks
    pattern = r'```json\n(.*?)\n```'

    # Find all matches in the input string
    matches = re.findall(pattern, input_string, re.DOTALL)

    json_objects = []

    for match in matches:
        try:
            # Parse the JSON string into a Python object
            json_obj = json.loads(match.strip())
            json_objects.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
            print(f"Problematic JSON string: {match}")
            continue

    return json_objects


# 5. Construct LLM Client

In [91]:
github_token=userdata.get('GITHUB_TOKEN')

endpoint = "https://models.github.ai/inference"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(github_token),
)

# 6. Normalize and Compare Answer

In [92]:
def find_name_column_index(answer):
    """Helper function to find the name column index in the header"""
    if 'header' in answer:
        try:
            return answer['header'].index('name')
        except (ValueError, IndexError):
            return None
    return None

def normalize_answer(answer, for_comparison=False):
    """
    Unified function to normalize answers for either counting or comparison.

    Args:
        answer: The answer to normalize
        for_comparison: If True, returns normalized rows for comparison.
                       If False, returns JSON string for counting duplicates.
    """
    if isinstance(answer, dict) and 'rows' in answer:
        name_idx = find_name_column_index(answer)

        if for_comparison:
            # For comparison: normalize all values and sort rows
            normalized_rows = []
            for row in answer['rows']:
                if name_idx is not None:
                    # Use the found name column index
                    normalized_row = []
                    for i, value in enumerate(row):
                        normalized_row.append(str(value).replace(" ", "").lower())
                    normalized_rows.append(tuple(normalized_row))
                else:
                    # Fallback: normalize all values
                    normalized_rows.append(tuple(str(val).replace(" ", "").lower() for val in row))

            # Sort rows by name for consistent comparison
            normalized_rows.sort()
            return normalized_rows
        else:
            # For counting: create dictionary keyed by name
            rows_by_name = {}
            for row in answer['rows']:
                if name_idx is not None:
                    name = row[name_idx]
                    rows_by_name[name] = tuple(row)  # Convert to tuple for hashability
                else:
                    # If no name column found, use second column (common pattern) or whole row
                    if len(row) > 1:
                        rows_by_name[row[1]] = tuple(row)
                    else:
                        rows_by_name[str(row)] = tuple(row)
            return json.dumps(rows_by_name, sort_keys=True)
    else:
        if for_comparison:
            return str(answer).replace(" ", "").lower()
        else:
            return str(answer)



# 7. Call LLM in loop to get samples

In [93]:
def generate_and_extract_answers(client, messages, num_samples, temperature, top_p, model_name):
    """
    Generate multiple samples and extract final answers from the responses.

    Args:
        client: The API client object
        messages: List of messages for the conversation
        num_samples: Number of samples to generate
        temperature: Temperature for sampling
        top_p: Top-p for sampling
        model_name: Name of the model to use

    Returns:
        list: List of extracted final answers
    """
    final_answers = []  # Collect all final answers


    for i in range(num_samples):
        print(f"\n📝 Generating sample {i+1}/{num_samples}...")
        # time.sleep(1)
        response = client.complete(
            messages=messages,
            temperature=temperature,
            top_p=top_p,
            model=model_name,
        )
        try:
          answer = response.choices[0].message.content
          generator_output_extracted = extract_final_answer_regex(answer)
          print(f"✅ Received response for sample {i+1}")

          if generator_output_extracted is None:
            print(f"❌ Warning: Could not extract answer from sample(Index: {i})")

          else:
            clean_generator_output_extracted = generator_output_extracted.lower().replace(" ", "")

            json_object_generator_output_extracted = extract_json_from_string(clean_generator_output_extracted)

            final_answers.append(json_object_generator_output_extracted[0])

        except Exception as e:
            print(f"Error processing sample {i+1}: {e}")

    return final_answers

# 8. Get final answer

In [94]:
def analyze_and_display_answers(final_answers, num_samples):
    if not final_answers:
        return

    # Convert dicts to JSON strings for counting
    json_strings = [json.dumps(answer, sort_keys=True) for answer in final_answers]
    counter = Counter(json_strings)

    # Get most common JSON string
    most_common_json, count = counter.most_common(1)[0]

    # Find the original dict that matches
    for answer in final_answers:
        if json.dumps(answer, sort_keys=True) == most_common_json:
            most_common_dict = answer
            break

    print(f"Most common answer (appears {count} times):")

    return most_common_dict, count

# 9. Check Correct Answer

In [95]:
def compare_answers(correct_answer, most_common_answer, final_answers):
    """
    Compare the most common answer and all final answers against the correct answer.

    Args:
        correct_answer: The ground truth correct answer
        most_common_answer: The most frequently occurring answer
        final_answers: List of all answers to check

    Returns:
        dict: Comparison results including matches and normalized answers
    """

    # Normalize the correct answer
    correct_normalized = normalize_answer(correct_answer, for_comparison=True)

    # Normalize the most common answer
    most_common_normalized = normalize_answer(most_common_answer, for_comparison=True)


    # Check if most common answer matches correct answer
    most_common_matches = most_common_normalized == correct_normalized

    # print(f"{'='*60}")
    # print(f"ans:{most_common_normalized}")
    # print(f"cor:{correct_normalized}")
    # print(f"{'='*60}")

    if most_common_matches:
        print("✅ SUCCESS: The most common answer is correct!")
        matching_answers = [i for i, answer in enumerate(final_answers)
                          if normalize_answer(answer, for_comparison=True) == correct_normalized]
    else:
        print("❌ The most common answer does NOT match the correct answer")
        print("\nChecking if any answer in final_answers matches the correct answer...")

        # Check all answers for matches
        matching_answers = []
        for i, answer in enumerate(final_answers):
            answer_normalized = normalize_answer(answer, for_comparison=True)
            if answer_normalized == correct_normalized:
                matching_answers.append(i)

        if matching_answers:
            print(f"✅ FOUND {len(matching_answers)} MATCHING ANSWER(S) at indices: {matching_answers}")
            # print(f"First matching answer: {final_answers[matching_answers[0]]}")
        else:
            print("❌ No answers in final_answers match the correct answer")

    # Return results for programmatic use
    return {
        'most_common_matches': most_common_matches,
        'matching_answers_indices': matching_answers,
        'correct_normalized': correct_normalized,
        'most_common_normalized': most_common_normalized,
        'matching_answers': [final_answers[i] for i in matching_answers] if matching_answers else []
    }

In [96]:
# Run all together
def runSC(question, solution, num_samples,temperature,top_p,model_name):
  messages = [
      SystemMessage(system_prompt),
      UserMessage(user_prompt),
      AssistantMessage(assistant_message),
      UserMessage(question)
  ]

  final_answers = generate_and_extract_answers(client, messages, num_samples, temperature, top_p, model_name)

  most_common_answer,count = analyze_and_display_answers(final_answers,num_samples)
  compare_result = compare_answers(solution, most_common_answer, final_answers)
  return {
      "most_common_answer": most_common_answer,
      "most_common_matches": compare_result["most_common_matches"],
      "final_answers": final_answers
  }

In [97]:
# @title


def load_puzzle(ds_size):
  # Or, use the token from Colab secrets
  hf_token = userdata.get('HF_TOKEN')
  dataset = load_dataset("allenai/ZebraLogicBench-private", "grid_mode", split="test", token=hf_token)
  grid_size = "4*4"

  filtered_dataset = dataset.filter(lambda x: x["size"] == grid_size).select(range(ds_size)).shuffle(seed=42)
  return filtered_dataset
  # display(filtered_dataset)
  # https://huggingface.co/datasets/allenai/ZebraLogicBench-private
  # Columns: id, size, puzzle, solution

In [98]:
def main(num_samples=1,ds_size = 15):
  is_correct_list = []
  filtered_dataset=load_puzzle(ds_size)


  # Get total number of items for progress tracking
  total_items = len(filtered_dataset)
  print(f"Starting processing of {total_items} items...")

  model_name = "openai/gpt-4.1-nano" # Choose not too smart model to see the diversity (phi-4,openai/gpt-4.1,deepseek/DeepSeek-R1-0528,xai/grok-3,openai/gpt-4.1-mini,deepseek/DeepSeek-V3-0324)
  temperature=1.0 # Choose 1.0 for the most diverse answers
  top_p=1.0 # Choose 1.0 for the most diverse answers

  for index, item in enumerate(filtered_dataset):
      # Calculate progress percentage
      progress = (index + 1) / total_items * 100


      puzzle_id = item["id"]
      puzzle_input = item["puzzle"]

      solution = item["solution"]

      print(f"\n--- Processing item {puzzle_id}:{index + 1}/{total_items} ({progress:.1f}%) ---")
      solution_normalized = normalize_answer(solution, for_comparison=True)
      full_question=f"{puzzle_input}\n## Headers\n\"header\":{item["solution"]['header']}"
      results = runSC(full_question,solution,num_samples,temperature,top_p,model_name)
      print(f"{'='*50}")
      is_correct_list.append(results['most_common_matches'])
      # delay/sleep 1 sec
      time.sleep(1)

  # Calculate final statistics
  correct_count = sum(1 for item in is_correct_list if item)
  accuracy = correct_count / len(is_correct_list) * 100 if is_correct_list else 0

  print(f"\n{'='*50}")
  print(f"✅ Iteration complete! Processed {len(is_correct_list)} items")
  print(f"📊 Results: {correct_count}/{len(is_correct_list)} correct ({accuracy:.1f}% accuracy)")
  print(f"{'='*50}")
  return results

In [ ]:
results_of_12=main(num_samples=10)

In [ ]:
results_of_1=main(num_samples=1)